# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [ ]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise and vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br />
 &emsp;| - audio1.wav<br />
 &emsp;| - audio2.wav<br />
 &emsp;| - audio3.wav<br />
  ...<br />
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`:

<span style="color:purple;font-size:15px">
# metadata.csv <br />
audio1|This is my sentence. <br />
audio2|This is maybe my sentence. <br />
audio3|This is certainly my sentence. <br />
audio4|Let this be your sentence. <br />
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br />
&emsp;| <br />
&emsp;| -> metadata.csv<br />
&emsp;| -> /wavs<br />
&emsp;&emsp;| -> audio1.wav<br />
&emsp;&emsp;| -> audio2.wav<br />
&emsp;&emsp;| ...<br />
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br />

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [ ]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


In [ ]:
# Download and extract LJSpeech dataset.

!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [ ]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [ ]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [ ]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [ ]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [ ]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [ ]:
trainer.fit()

#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint.

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

## 📣 Listen to the synthesized wave 📣

In [7]:
!pip install cssselect
!pip install lxml


## 🎉 Congratulations! 🎉 You now have trained your first TTS model!
Follow up with the next tutorials to learn more advanced material.

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Finished page 3. Found 20 titles.
Fetching page 11...
Finished page 9. Found 20 titles.
Finished page 6. Found 20 titles.
Finished page 10. Found 20 titles.
Finished page 8. Found 20 titles.
Finished page 4. Found 20 titles.


/usr/local/lib/python3.11/dist-packages/bs4/element.py:1300: RuntimeWarning: coroutine 'main' was never awaited
  u = str.__new__(cls, value)


Finished page 7. Found 20 titles.
Finished page 2. Found 20 titles.
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Finished page 1. Found 20 titles.
Finished page 5. Found 20 titles.
Fetching page 19...
Fetching page 20...
Finished page 16. Found 20 titles.
Fetching page 21...
Finished page 14. Found 20 titles.
Finished page 15. Found 20 titles.
Finished page 12. Found 20 titles.
Finished page 13. Found 20 titles.
Finished page 11. Found 20 titles.
Finished page 18. Found 20 titles.
Finished page 17. Found 20 titles.
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Finished page 20. Found 20 titles.
Finished page 19. Found 20 titles.
Fetching page 29...
Fetching page 30...
Finished page 22. Found 20 titles.
Fetching page 31...
Finished page 25. Found 20 titles.
Finished page 21. Found 20 titles.
Finished 

In [17]:
import asyncio
import httpx
import lxml.html
import json
import logging
import urllib.parse
import nest_asyncio

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

BASE_URL = "https://kurdfilm.krd"
BASE_PAGE_URL = "https://kurdfilm.krd/va/m?_token=2wrbDq29KA94sDjR15hRDTOXyf9DCeJchMCHi8fE&type=all&genre=genre&releasedate=year"

START_PAGE = 1
END_PAGE = 156

async def fetch_page(client: httpx.AsyncClient, page_num: int):
    """Fetches a single page and returns the HTML content."""
    url = f"{BASE_PAGE_URL}&page={page_num}"
    logging.info(f"Fetching page {page_num}: {url}")
    try:
        response = await client.get(url, timeout=30.0)
        response.raise_for_status()
        logging.info(f"Successfully fetched page {page_num}")
        return response.text
    except httpx.RequestError as e:
        logging.error(f"An HTTP error occurred while fetching page {page_num}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred while fetching page {page_num}: {e}")
        return None


def parse_movie_data(html_content: str):
    """Parses HTML content and extracts movie details."""
    movies = []
    if not html_content:
        return movies

    try:
        tree = lxml.html.fromstring(html_content)
        movie_elements = tree.xpath('//div[contains(@class, "row")]/div[contains(@class, "col-6")]')

        for movie_el in movie_elements:
            try:
                link_el = movie_el.xpath('.//a[contains(@class, "stretched-link")]')[0]
                movie_link = urllib.parse.urljoin(BASE_URL, link_el.get('href'))

                img_el = link_el.xpath('.//img[contains(@class, "img-fluid")]')[0]
                movie_img_url = urllib.parse.urljoin(BASE_URL, img_el.get('src'))

                title_el = movie_el.xpath('.//div[contains(@class, "product-title")]/a')[0]
                movie_title = title_el.text_content().strip()

                genre_el = movie_el.xpath('.//div[contains(@class, "product-meta")]/span[@class="text-gray-1300"]')
                genre = genre_el[0].text_content().strip() if genre_el else None

                rating = None
                if genre_el:
                    text_content = genre_el[0].text_content().strip()
                    if '10/' in text_content:
                        rating_part = text_content.split('10/')[1].split('<i')[0].split(' ')[0]
                        try:
                            rating = float(rating_part)
                        except ValueError:
                            rating = rating_part

                movies.append({
                    "title": movie_title,
                    "link": movie_link,
                    "image_url": movie_img_url,
                    "genre": genre,
                    "rating": rating
                })

            except Exception as e:
                logging.error(f"Error parsing a movie element on the page: {e}", exc_info=True)

    except Exception as e:
        logging.error(f"Error parsing HTML content: {e}", exc_info=True)

    return movies


async def main():
    """Main function to orchestrate fetching and parsing multiple pages."""
    all_movies = []
    pages_to_fetch = list(range(START_PAGE, END_PAGE + 1))

    async with httpx.AsyncClient() as client:
        tasks = [fetch_page(client, page) for page in pages_to_fetch]
        html_contents = await asyncio.gather(*tasks)

    for page_num, html_content in zip(pages_to_fetch, html_contents):
        if html_content:
            logging.info(f"Parsing data from page {page_num}")
            movies_on_page = parse_movie_data(html_content)
            all_movies.extend(movies_on_page)
            logging.info(f"Found {len(movies_on_page)} movies on page {page_num}. Total movies collected: {len(all_movies)}")
        else:
            logging.warning(f"Skipping parsing for page {page_num} due to fetch error.")

    try:
        with open("kurdfilm_movies.json", "w", encoding="utf-8") as f:
            json.dump(all_movies, f, ensure_ascii=False, indent=4)
        logging.info(f"Successfully scraped {len(all_movies)} movies and saved to kurdfilm_movies.json")
    except Exception as e:
        logging.error(f"Error saving data to JSON file: {e}")


if __name__ == "__main__":
    logging.info(f"Starting scraper for pages {START_PAGE} to {END_PAGE}...")
    nest_asyncio.apply()  # Apply nest_asyncio patch
    asyncio.run(main())
    logging.info("Scraping finished.")

In [14]:
import httpx
import asyncio
import json
from lxml import html

BASE_URL = "https://kurdfilm.krd/va/t?type=all&page={}"  # Pages 1-20
SITE_ROOT = "https://kurdfilm.krd"

async def fetch_page(client, page_num):
    url = BASE_URL.format(page_num)
    try:
        response = await client.get(url, timeout=15)
        response.raise_for_status()
        return response.text
    except httpx.HTTPError as e:
        print(f"Failed to fetch page {page_num}: {e}")
        return None

def extract_tvshows(html_content):
    tree = html.fromstring(html_content)
    tvshows = []

    for product in tree.xpath('//div[contains(@class, "product")]'):
        title_el = product.xpath('.//div[contains(@class, "product-title")]/a')
        img_el = product.xpath('.//div[contains(@class, "product-image")]//img')

        if not title_el or not img_el:
            continue

        title = title_el[0].text_content().strip()
        link = title_el[0].get("href")
        image = img_el[0].get("src")

        if not link.startswith("http"):
            link = SITE_ROOT + link
        if not image.startswith("http"):
            image = SITE_ROOT + image

        tvshows.append({
            "title": title,
            "link": link,
            "image": image
        })
    return tvshows

async def main():
    tvshows_data = []
    async with httpx.AsyncClient(http2=True) as client:
        tasks = [fetch_page(client, i) for i in range(1, 21)]
        pages = await asyncio.gather(*tasks)

        for content in pages:
            if content:
                tvshows_data.extend(extract_tvshows(content))

    with open("tvshows.json", "w", encoding="utf-8") as f:
        json.dump(tvshows_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Scraped {len(tvshows_data)} TV shows and saved to 'tvshows.json'")

if __name__ == "__main__":
    asyncio.run(main())

✅ Scraped 519 TV shows and saved to 'tvshows.json'
